In [ ]:
!pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.7/427.7 kB 30.3 MB/s eta 0:00

In [1]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import seaborn as sn

ModuleNotFoundError: No module named 'mesa'

In [ ]:
from google.colab import  files
uploaded = files.upload()

In [ ]:
class Cell():
    def __init__(self, x, y, wall):
        self.pos = (x, y)

        if wall[0] == '1': self.up = True
        else: self.up = False
        if wall[1] == '1': self.left = True
        else: self.left = False
        if wall[2] == '1': self.down = True
        else: self.down = False
        if wall[3] == '1': self.right = True
        else: self.right = False

        self.alert = 0 # 1- false alarm    2 - victim
        self.fire = 0 # 1 - smoke    2 - fire
        self.door = [] # pos of the connected cell with a door & if is open    [] if no door
        self.entrance = False # True if the current cell is a access point to the structure

In [ ]:
!rm mapa.txt

In [ ]:
class FiremanAgent(Agent):
    def __init__(self, unique_id, model, point=None):
        super().__init__(unique_id, model)
        self.point = point

    def step(self):
        if self.point is not None:
            path = self.dijkstra(self.pos, self.model.interestPoints[self.point])[0]
            if path:
                self.model.grid.move_agent(self, path[0])
        else:
            # Encuentra el punto de fuego más cercano y muévete hacia él
            closest_fire_point = None
            min_distance = float('inf')
            for fire_point in self.model.firePoints:
                distance = self.dijkstra(self.pos, fire_point)[1]
                if distance < min_distance:
                    min_distance = distance
                    closest_fire_point = fire_point
            if closest_fire_point:
                path = self.dijkstra(self.pos, closest_fire_point)[0]
                if path:
                    self.model.grid.move_agent(self, path[0])

    def dijkstra(self, start, end):
        dijkstraMap = {}
        path = []
        for x in range(self.model.grid.width):
            for y in range(self.model.grid.height):
                dijkstraMap[(x, y)] = {"previousCell": None, "steps": None}
        if start in dijkstraMap:
            dijkstraMap[start]["steps"] = 0
            dijkstraMap[start]["previousCell"] = start
            queue = [start]
            while len(queue) > 0:
                for neighbor in self.model.grid.get_neighbors(queue[0], moore=False):
                    if dijkstraMap[neighbor]["steps"] is None:
                        dijkstraMap[neighbor]["steps"] = self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]
                        dijkstraMap[neighbor]["previousCell"] = queue[0]
                        queue.append(neighbor)
                    elif (dijkstraMap[neighbor]["steps"] > self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]):
                        dijkstraMap[neighbor]["steps"] = (self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"])
                        dijkstraMap[neighbor]["previousCell"] = queue[0]
                        queue.append(neighbor)
                queue.pop(0)
            cell = end
            while cell != start:
                if cell in dijkstraMap:
                    path.insert(0, cell)
                    cell = dijkstraMap[cell]["previousCell"]
            return path, dijkstraMap[end]["steps"]
        else:
            print("No existe esa celda de inicio")
            return path, dijkstraMap[end]["steps"]

    def calculateSteps(self, start, end):
        # Aquí puedes calcular el costo de moverse de start a end
        # Por ahora, asumimos que el costo es siempre 1
        return 1


In [ ]:
class MapModel(Model):
    def __init__(self, num_agents, cells):
        super().__init__()
        self.interestPoints = interestPoints
        self.firePoints = firePoints
        self.structural_Damage_Left = 24
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, True)
        self.cells = cells
        self.schedule = RandomActivation(self)
        self.running = True
        empty_positions = [(x, y) for x in range(self.width) for y in range(self.height)]
        for i in range(self.num_agents):
            random_pos = self.random.choice(empty_positions)
            a = FiremanAgent(i, self)  # No asignamos punto inicialmente
            empty_positions.remove(random_pos)
            self.schedule.add(a)
            self.grid.place_agent(a, random_pos)

    def step(self):
        # Asigna los 3 agentes más cercanos a los primeros 3 puntos de interés
        closestAgentToPoint = []
        for i in range(min(3, len(self.interestPoints))):
            point = self.interestPoints[i]
            minSteps = float('inf')
            closestAgent = None
            for agent in self.schedule.agents:
                if agent.point is None:  # Solo considera agentes no asignados
                    steps = agent.dijkstra(agent.pos, point)[1]
                    if steps < minSteps:
                        minSteps = steps
                        closestAgent = agent
            if closestAgent:
                closestAgentToPoint.append((closestAgent, minSteps, i))
        closestAgentToPoint.sort(key=lambda x: x[1])
        for agent, steps, point_index in closestAgentToPoint:
            agent.point = point_index

        # El resto de los agentes se dirigen a los puntos de fuego
        self.schedule.step()



In [2]:
with open('mapa.txt', 'r') as map:
    text = map.read()

walls = []
for i in range(8):
    for j in range(6):
        new_wall = text[:4]
        walls.append(new_wall)
        text = text[5:]

alerts = []
for i in range(3):
    pos_alert_x = text[0]
    pos_alert_y = text[2]
    pos_alert_state = text[4]
    text = text[6:]
    alerts.append( (pos_alert_x, pos_alert_y, pos_alert_state) )

fires = []
for i in range(10):
    pos_fire_x = text[0]
    pos_fire_y = text[2]
    text = text[4:]
    fires.append( (pos_fire_x, pos_fire_y) )

doors = []
for i in range(8):
    pos_doorA_x = text[0]
    pos_doorA_y = text[2]
    pos_doorB_x = text[4]
    pos_doorB_y = text[6]
    text = text[8:]
    doors.append( ( (pos_doorA_x, pos_doorA_y), (pos_doorB_x, pos_doorB_y) ) )

exits = []
for i in range(4):
    pos_exit_x = text[0]
    pos_exit_y = text[2]
    text = text[4:]
    exits.append( (pos_exit_x, pos_exit_y) )

cells = []
for i in range(6):
    for j in range(8):
        w = walls[0]
        del walls[0]

        c = Cell(i + 1,j + 1,w)
        cells.append(c)

        if (str(i + 1), str(j + 1), 'v') in alerts:
            c.alert = 2
        elif (str(i + 1), str(j + 1), 'f') in alerts:
            c.alert = 1

        if (str(i + 1), str(j + 1)) in fires:
            c.fire = 2

        for d in doors:
            if (str(i + 1), str(j + 1)) == d[0]:
                c.door = d[1]
            elif (str(i + 1), str(j + 1)) == d[1]:
                c.door = d[0]

        if (str(i + 1), str(j + 1)) in exits:
            c.entrance = True

        print(f"{c.pos}: {c.up} - {c.left} - {c.down} - {c.right}   A: {c.alert}   F: {c.fire}   D: {c.door}    E: {c.entrance}")


NameError: name 'MapModel' is not defined